In [3]:
|!pip install jsonlines
!pip install pandas
!pip install transformers
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
Looking in indexes: https://download.pytorch.org/whl/cpu
     |████████████████████████████████| 195.4 MB 277 bytes/s ta 0:00:01  |███████████████▌                | 95.0 MB 5.0 MB/s eta 0:00:21
     |████████████████████████████████| 24.9 MB 28 kB/s s eta 0:00:01    |█                               | 706 kB 23.3 MB/s eta 0:00:02
     |████████████████████████████████| 4.1 MB 20 kB/s  eta 0:00:01


In [3]:
import jsonlines 
import pandas as pd
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
import json

In [4]:
rutas = ["/home/julieta/LanAn/aguadiana4-3_tweetsResCont.txt"]

In [5]:
df = pd.DataFrame()
for ruta in rutas:
    df = pd.concat([df, pd.read_json(ruta, lines=True)], ignore_index=True)
df

,_type,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,...,bookmarkCount,pinned,editState,content,outlinks,outlinksss,tcooutlinks,tcooutlinksss,username,_snscrape
0,snscrape.modules.twitter.Tweet,https://twitter.com/cjauglez/status/1643767467...,2023-04-06 00:07:58+00:00,@aguadiana Don canguro,@aguadiana Don canguro,1643767467494481925,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,0,NaN,{'_type': 'snscrape.modules.twitter.EditState'...,@aguadiana Don canguro,[],,[],,cjauglez,0.6.2.20230321.dev50+g0d824ab
1,snscrape.modules.twitter.Tweet,https://twitter.com/Roberto61609862/status/164...,2023-04-05 02:37:10+00:00,@aguadiana El cambio https://t.co/qX6OreFnCW,@aguadiana El cambio https://t.co/qX6OreFnCW,1643442625922383873,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,0,NaN,{'_type': 'snscrape.modules.twitter.EditState'...,@aguadiana El cambio https://t.co/qX6OreFnCW,[],,[],,Roberto61609862,0.6.2.20230321.dev50+g0d824ab
2,snscrape.modules.twitter.Tweet,https://twitter.com/cuchoalbert/status/1643387...,2023-04-04 22:56:15+00:00,@aguadiana El cambio https://t.co/mNXSDlGinx,@aguadiana El cambio https://t.co/mNXSDlGinx,1643387032310370304,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,0,NaN,{'_type': 'snscrape.modules.twitter.EditState'...,@aguadiana El cambio https://t.co/mNXSDlGinx,[],,[],,cuchoalbert,0.6.2.20230321.dev50+g0d824ab
3,snscrape.modules.twitter.Tweet,https://twitter.com/Franky61529058/status/1643...,2023-04-04 21:29:52+00:00,@aguadiana Para cuando te estas alineando con ...,@aguadiana Para cuando te estas alineando con ...,1643365291630510085,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,2,1,...,0,NaN,{'_type': 'snscrape.modules.twitter.EditState'...,@aguadiana Para cuando te estas alineando con ...,[],,[],,Franky61529058,0.6.2.20230321.dev50+g0d824ab
4,snscrape.modules.twitter.Tweet,https://twitter.com/pavzqzo/status/16431440579...,2023-04-04 06:50:46+00:00,@aguadiana gordito cochino te usaron. el deal ...,@aguadiana gordito cochino te usaron. el deal ...,1643144057953173507,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,0,NaN,{'_type': 'snscrape.modules.twitter.EditState'...,@aguadiana gordito cochino te usaron. el deal ...,[],,[],,pavzqzo,0.6.2.20230321.dev50+g0d824ab
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,snscrape.modules.twitter.Tweet,https://twitter.com/GhfMosh/status/16430374269...,2023-04-03 23:47:03+00:00,@aguadiana Pues la verdad el no tiene nada de ...,@aguadiana Pues la verdad el no tiene nada de ...,1643037426967891970,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,0,NaN,{'_type': 'snscrape.modules.twitter.EditState'...,@aguadiana Pues la verdad el no tiene nada de ...,[],,[],,GhfMosh,0.6.2.20230321.dev50+g0d824ab
103,snscrape.modules.twitter.Tweet,https://twitter.com/DragonVladimir/status/1642...,2023-04-03 17:18:27+00:00,@aguadiana Sólo eres un prianista corrupto y c...,@aguadiana Sólo eres un prianista corrupto y c...,1642939634316460050,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,...,0,NaN,{'_type': 'snscrape.modules.twitter.EditState'...,@aguadiana Sólo eres un prianista corrupto y c...,[],,[],,DragonVladimir,0.6.2.20230321.dev50+g0d824ab
104,snscrape.modules.twitter.Tweet,https://twitter.com/Mysteriodf77/status/164293...,2023-04-03 17:17:46+00:00,@aguadiana El viejo ridículo que no se quita e...,@aguadiana El viejo ridículo que no se quita e...,1642939462844923906,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,0,NaN,{'_type': 'snscrape.modules.twitter.EditState'...,@aguadiana El viejo ridículo que no se quita e...,[],,[],,Mysteriodf77,0.6.2.20230321.dev50+g0d824ab
105,snscrape.modules.twitter.Tweet,https://twitter.com/ramonav11/status/164293075...,2023-04-03 16:43:10+00:00,@aguadiana Guadiana y Mejía \n\nLa división fa...,@aguadiana Guadiana y Mejía \n\nLa división fa...,1642930752051372032,"{'_type': 'snscr

In [6]:
model_path = "daveni/twitter-xlm-roberta-emotion-es"
emotion_analysis = pipeline("text-classification", framework="pt", model=model_path, tokenizer=model_path)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [7]:
data = []

In [11]:
for i,row in df.iterrows():
    name = row["inReplyToUser"]
    name= str(name)
    name = name.replace("'", "\"")
    name = name.replace("None", "null")
    try:
        objeto_name = json.loads(name)
        try:
            figura = objeto_name["username"]
            try:
                json_res = emotion_analysis(row["rawContent"])
                #obj_res = json.loads(json_res[0])
                #emocion = obj_res["label"]
                #porcentaje = obj_res["score"]
                emocion = "emocion"
                porcentaje="30%"
                data.append ({"twitt" : row["rawContent"], "emocion" :emocion, "porcentaje" : porcentaje, "figura": figura, "fecha" : row["date"]})
            except :
                print("Error:Modelo")
        except :
            print("Error:FIGURA")
    
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", str(e))
    

In [10]:
df_final = pd.DataFrame(data)
df_final

,twitt,emocion,porcentaje,figura,fecha
0,@aguadiana Don canguro,emocion,30%,aguadiana,2023-04-06 00:07:58+00:00
1,@aguadiana El cambio https://t.co/qX6OreFnCW,emocion,30%,aguadiana,2023-04-05 02:37:10+00:00
2,@aguadiana El cambio https://t.co/mNXSDlGinx,emocion,30%,aguadiana,2023-04-04 22:56:15+00:00
3,@aguadiana Para cuando te estas alineando con ...,emocion,30%,aguadiana,2023-04-04 21:29:52+00:00
4,@aguadiana gordito cochino te usaron. el deal ...,emocion,30%,aguadiana,2023-04-04 06:50:46+00:00
...,...,...,...,...,...
102,@aguadiana Pues la verdad el no tiene nada de ...,emocion,30%,aguadiana,2023-04-03 23:47:03+00:00
103,@aguadiana Sólo eres un prianista corrupto y c...,emocion,30%,aguadiana,2023-04-03 17:18:27+00:00
104,@aguadiana El viejo ridículo que no se quita e...,emocion,30%,aguadiana,2023-04-03 17:17:46+00:00
105,@aguadiana Guadiana y Mejía \n\nLa división fa...,emocion,30%,aguadiana,2023-04-03 16:43:10+00:00
